In [2]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [3]:



hp_data = pd.read_csv('hp_all_comments.csv')
hp_data.comment=hp_data.comment.astype(str)
hp_data=hp_data['comment']


hp_data_lst = hp_data.tolist()

In [4]:
with open('hp-500k-tok.pickle', 'rb') as handle:
    hp_tok = pickle.load(handle)
print('vocab size: {}'.format(len(hp_tok.index_word)))

vocab size: 185447


start = time.time()
hp_500k_tok = Tokenizer(num_words=50000,oov_token='<unk>')
hp_500k_tok.fit_on_texts(hp_data_lst[:500000])
with open('hp-500k-tok.pickle', 'wb') as handle:
    pickle.dump(hp_500k_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('bb tokenization time: {}'.format(time.time() - start))

In [5]:


train_data = hp_data_lst[:2000000]

seqs = hp_tok.texts_to_sequences(train_data)
less_20_seqs = list(filter(lambda x: len(x) <20, seqs))[:1000000]

In [6]:
MAXLEN = 19
VOCAB = int(len(hp_tok.index_word))
SEED = 42

sequences = pad_sequences(less_20_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [7]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(LSTM(500))



model.add(Dense(VOCAB, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           55634100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 185447)            92908947  
Total params: 150,145,047
Trainable params: 150,145,047
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [9]:

model_json = model.to_json()
with open("models/hp-model-v4.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/huffpo-v4/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('huffpo-modelv4'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.005, epochs=200, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 845750 samples, validate on 4250 samples
Epoch 1/200


ResourceExhaustedError: OOM when allocating tensor with shape[500,185447] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/Square_4}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
